In [4]:
import glob,pandas as pd

df = pd.concat([pd.read_csv(filename,header=None,names=["name","gender","count"]) for filename in glob.glob("./names/yob*.txt")], axis=0)

names = df.groupby(["name","gender"]).sum().reset_index()
names = names.merge(names.groupby("name")["count"].sum(),on="name",suffixes=("","_total"))
names["prob"] = (names["count"]/names.count_total).round(2)
names = names[names.gender=="M"].merge(names[names.gender=="F"],how="outer",on=["name","count_total"],suffixes=("_m","_f"))
names.loc[names.count_m.isna(),"prob_m"] = 1 - names[names.count_m.isna()].prob_f
names.count_f = names.count_f.fillna(0)
names.count_m = names.count_m.fillna(0)
names = names.drop(columns=["gender_m","gender_f","prob_f"])
names = names.rename({"prob_m":"is.man"},axis=1)
names

,name,count_m,count_total,is.man,count_f
0,Aaban,127.0,127,1.0,0.0
1,Aabid,16.0,16,1.0,0.0
2,Aabir,19.0,19,1.0,0.0
3,Aadam,333.0,333,1.0,0.0
4,Aadan,136.0,136,1.0,0.0
...,...,...,...,...,...
102444,Zyrielle,0.0,32,0.0,32.0
102445,Zyrihanna,0.0,50,0.0,50.0
102446,Zyriyah,0.0,16,0.0,16.0
102447,Zyva,0.0,47,0.0,47.0


In [5]:
from collections import defaultdict
infer_gender = defaultdict(lambda :pd.NA)
names.name = names.name.str.lower()
infer_gender.update(names[["name","is.man"]].set_index("name").to_dict()["is.man"])

In [6]:
bgg_Person = pd.read_csv('./scraped/bgg_Person.csv')
bgg_Person

,bgg_id,name
0,1,Karl-Heinz Schmiel
1,2,Reiner Knizia
2,3,(Uncredited)
3,4,Sid Sackson
4,5,Jean du Poël
...,...,...
53336,157370,Admir Tihić
53337,157377,Hippo Design
53338,157378,Ratchanon Wanichsombat
53339,157379,Phopkrit Tiyarattanachai


In [7]:
first_names = bgg_Person.name.str.split(pat=r"[^a-zA-Z]",regex=True)
inferred_genders = first_names.explode().str.lower().map(infer_gender).dropna().groupby(level=0).apply(list).map(lambda x:x[0])
bgg_augmented = bgg_Person.join(inferred_genders.rename("is.man"),how="outer")
bgg_augmented.loc[(bgg_augmented["is.man"] < 0.9)&(bgg_augmented["is.man"] > 0.1)].loc[:,"is.man"] = pd.NA
bgg_augmented["is.man"] = bgg_augmented["is.man"].map(lambda x :x>0.5,na_action="ignore").fillna(-1).astype(int)
bgg_augmented.to_csv("bgg_person_gendered.csv",index=False)

In [10]:
bgg_augmented["is.man"].value_counts()

is.man
 1    37459
 0    10569
-1     5313
Name: count, dtype: int64

In [1]:
bgg_augmented

NameError: name 'bgg_augmented' is not defined